<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-и-изучение-файла" data-toc-modified-id="Загрузка-и-изучение-файла-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка и изучение файла</a></span></li><li><span><a href="#Деление-данных-на-выборки" data-toc-modified-id="Деление-данных-на-выборки-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Деление данных на выборки</a></span></li><li><span><a href="#Исследование-моделей" data-toc-modified-id="Исследование-моделей-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Исследование моделей</a></span></li><li><span><a href="#Проверка-моделей-на-тестовой-выборке" data-toc-modified-id="Проверка-моделей-на-тестовой-выборке-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка моделей на тестовой выборке</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Вывод</a></span></li></ul></div>

# Описание проекта - Рекомендация тарифов

Оператор мобильной связи «Мегалайн» хочет построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: `«Смарт»` или `«Ультра»`.  

В распоряжении имеются данные `users_behavior.csv` о поведении клиентов, которые уже перешли на эти тарифы.  

Требуется построить модель с максимально большим значением `accuracy` для задачи классификации, которая выберет подходящий тариф.

## Загрузка и изучение файла 

Импортируем все необходимые библиотеки и модули

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

Файл с данными называется `users_behavior.csv`. Загрузим и изучим его.

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')

Выведем первые пять строк датафрейма, изучим названия колокнок и посмотрим размер с помощью `.shape`

In [3]:
print(df.head())
print(df.shape)

   calls  minutes  messages   mb_used  is_ultra
0   40.0   311.90      83.0  19915.42         0
1   85.0   516.75      56.0  22696.96         0
2   77.0   467.66      86.0  21060.45         0
3  106.0   745.53      81.0   8437.39         1
4   66.0   418.74       1.0  14502.75         0
(3214, 5)


Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:

`сalls` — количество звонков,  
`minutes` — суммарная длительность звонков в минутах,  
`messages` — количество sms-сообщений,  
`mb_used` — израсходованный интернет-трафик в Мб,  
`is_ultra` — каким тарифом пользовался в течение месяца («Ультра» — `1`, «Смарт» — `0`).  

## Деление данных на выборки

С помощью `train_test_split` разделим исходные данные на обучающую `df_train`, валидационную `df_valid` и тестовую `df_test` выборки. Размеры тестового и валидационного наборов обычно равны.  
Исходные данные разбивают в соотношении `3:1:1`. Размер укажем в `test_size=0.40`, а для разделения `df_valid` разделим пополам `test_size=0.50`.
Зафиксируем псевдослучайность для алгоритма обучения укажем параметр `random_state=12345`<br>
Выведем на экран размеры выборок с помощью `.shape`

In [4]:
df_train, df_valid = train_test_split(df, test_size=0.40, random_state=12345)
df_test, df_valid = train_test_split(df_valid, test_size=0.50, random_state=12345)
print(df_train.shape)
print(df_valid.shape)
print(df_test.shape)

(1928, 5)
(643, 5)
(643, 5)


Сохраним признаки каждой выборки `_train`, `_test` и `_valid` в отдельных переменных `features` и `target` 

In [5]:
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']

In [6]:
features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']

In [7]:
features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']

## Исследование моделей

Модель - `Дерево решений`, с помощью цикла попробуем разные значения одного параметра — максимальной глубины, `max_depth` и найдем `accuracy_score` 

In [8]:
best_model = None
best_result = 0
for depth in range(1, 6):
    # обучаем модель с заданной глубиной дерева
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    # обучаем модель
    model.fit(features_train, target_train)
    # считаем качество модели
    result = model.score(features_valid, target_valid) 
    if result > best_result:
        best_model = model
        best_result = result
        
print(f"Accuracy лучшей модели: {best_result}")
print(f"Наилучшая модель: \n{best_model}")

Accuracy лучшей модели: 0.7838258164852255
Наилучшая модель: 
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=5,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=12345, splitter='best')


Модель - `Случайный лес`, должна помочь улучшить результат предсказания и избежать переобучения. Чтобы управлять количеством деревьев в лесу, пропишем гиперпараметр `n_estimators`, кроме того найдем и лучшее значение для `max_depth`

In [9]:
best_model = None
best_result = 0
for est in range(1, 11):
    for depth in range(1, 6):
        # обучаем модель с заданным количеством деревьев
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        # обучаем модель на тренировочной выборке
        model.fit(features_train, target_train)
        # считаем качество модели на валидационной выборке
        result = model.score(features_valid, target_valid) 
        if result > best_result:
            # сохраним наилучшую модель
            best_model = model 
            # сохраним наилучшее значение метрики accuracy на валидационных данных
            best_result = result 

print(f"Accuracy наилучшей модели на валидационной выборке: {best_result}")
print(f"Наилучшая модель: \n{best_model}")


Accuracy наилучшей модели на валидационной выборке: 0.7993779160186625
Наилучшая модель: 
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=5, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=4,
                       n_jobs=None, oob_score=False, random_state=12345,
                       verbose=0, warm_start=False)


Модель - `Логистическая регрессия`, для постоянства результата зададим `random_state`, равный `12345`.

In [10]:
# Создадим модель логистической регрессии с параметром random_state=12345
model = LogisticRegression(random_state=12345) 

# Обучим модель на тренировочной выборке
model.fit(features_train, target_train) 

# Получим метрику качества модели на валидационной выборке
result = model.score(features_valid, target_valid) 

print(f"Accuracy модели логистической регрессии на валидационной выборке: {result}")

Accuracy модели логистической регрессии на валидационной выборке: 0.7402799377916018


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


## Проверка моделей на тестовой выборке

Проверим модели `DecisionTreeClassifier`, `RandomForestClassifier` и `LogisticRegression` на тестовой выборке и выведем `accuracy` моделей

Модель - Решающее дерево

In [11]:
model = DecisionTreeClassifier(random_state=12345, max_depth=3)
model.fit(features_train, target_train)
result = model.score(features_test, target_test)

print(f"Accuracy модели решающего дерева на тестовой выборке: {result}")

Accuracy модели решающего дерева на тестовой выборке: 0.7853810264385692


Модель - Случайный лес

In [12]:
model = RandomForestClassifier(random_state=12345, n_estimators=8, max_depth=5)
model.fit(features_train, target_train)
result = model.score(features_test, target_test)

print(f"Accuracy модели случайного леса на тестовой выборке: {result}")

Accuracy модели случайного леса на тестовой выборке: 0.7931570762052877


Модель - Логистическая регрессия

In [13]:
model = LogisticRegression(random_state=12345) 
model.fit(features_train, target_train) 
result = model.score(features_test, target_test)

print(f"Accuracy модели логистической регрессии на тестовой выборке: {result}")

Accuracy модели логистической регрессии на тестовой выборке: 0.7589424572317263


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


---

## Вывод

Исходный файл разделили на обучающую `df_train`, валидационную `df_valid` и тестовую `df_test` выборки а признаки были сохранены в соответствующх отдельных переменных `features` и `target`. <br>
Исследовали три модели с различными настройками и тестами чтобы найти наиболее точное значение для `accuracy` моделей.

При проверке все модели показали значение выше `0.75` на тестовой выборке<br> 
Самое высокое качество предсказания получилось у модели случайного леса, с гипперпараметрами `n_estimators=8` и `max_depth=5`. 

---